In [57]:
from nav_EDA import nav_df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


nav_df.head()




,index,date,100033,100034,100037,100038,100041,100042,100043,100044,100046,100047,100048,100051,100057,100058,100063,100064,100067,100068,100077,100078,100079,100080,100081,100082,100084,100085,100086,100087,100088,100089,100119,100120,100123,100124,100152,100153,100156,100171,...,153385,153386,153387,153388,153389,153390,153391,153392,153398,153399,153400,153401,153402,153403,153404,153405,153406,153407,153408,153409,153410,153411,153412,153413,153414,153415,153416,153417,153418,153419,153431,153432,153433,153434,153435,153438,153443,153444,153445,153451
0,0,06-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,07-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,08-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,09-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10-Apr-2020,NaN,NaN,NaN,NaN,NaN,-2.755585,-2.754775,NaN,NaN,-2.749929,-2.740088,-2.61965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nav_pct_df = nav_df
nav_pct_df.reset_index(inplace=True)
nav_pct_df.set_index('date', inplace=True)
for col in nav_pct_df.columns:
    nav_pct_df[col] = nav_df[col].pct_change()

nav_pct_df.head(20)

/var/folders/j3/btykbjss2l15b55qw91h85hc0000gn/T/ipykernel_56301/1381284010.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nav_pct_df.reset_index(inplace=True)
/var/folders/j3/btykbjss2l15b55qw91h85hc0000gn/T/ipykernel_56301/1381284010.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  nav_pct_df[col] = nav_pct_df[col].pct_change()


,100033,100034,100037,100038,100041,100042,100043,100044,100046,100047,100048,100051,100057,100058,100063,100064,100067,100068,100077,100078,100079,100080,100081,100082,100084,100085,100086,100087,100088,100089,100119,100120,100123,100124,100152,100153,100156,100171,100172,100173,...,153385,153386,153387,153388,153389,153390,153391,153392,153398,153399,153400,153401,153402,153403,153404,153405,153406,153407,153408,153409,153410,153411,153412,153413,153414,153415,153416,153417,153418,153419,153431,153432,153433,153434,153435,153438,153443,153444,153445,153451
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
06-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
08-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09-Apr-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10-Apr-2020,NaN,NaN,NaN,NaN,NaN,-2.755585,-2.754775,NaN,NaN,-2.749929,-2.740088,-2.619650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13-Apr-2020,-0.055123,-0.054666,-0.575943,-0.574032,NaN,0.761243,0.769659,NaN,NaN,0.683212,-9.080617,-9.998094,-0.099798,-0.099921,-0.336279,-0.337094,-0.142325,-0.178381,-1.799731,-1.804676,-1.804592,-0.085428,-0.033026,-0.032783,-1.121624,-1.118526,-1.121415,-1.061721,-1.060346,-1.068109,1.130370e-01,1.126086e-01,-1.182900,-1.181408,-0.221872,-0.221787,-0.050597,-0.175003,-0.169884,-0.174974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14-Apr-2020,169.930728,170.222214,890.279197,892.179531,-0.500033,-1.468270,-1.466885,-0.500033,-0.500033,-1.495477,-0.936005,-0.938360,738.200551,742.326659,245.406217,248.350690,28.717521,316.031658,508.330443,512.812644,511.881919,98.914077,139.723014,140.153416,557.761635,560.008264,559.210422,746490.999106,750242.382925,604962.089771,1.061798e+06,1.690114e+06,-711.428502,-712.939446,6037.125233,6042.433413,156.913958,306.853493,331.506724,332.853447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15-Apr-2020,-0.994189,-0.994199,-0.998876,-0.998879,-1.000000,-0.551015,-0.552253,-1.000000,-1.000000,-0.555567,-0.051059,-0.051978,-0.998636,-0.998644,-0.995943,-0.995991,-0.965485,-0.996861,-0.998031,-0.998048,-0.998045,-0.990029,-0.992920,-0.992941,-0.998204,-0.998212,-0.998209,-0.999999,-0.999999,-0.999998,-9.999991e-01,-9.999994e-01,-1.001407,-1.001404,-0.999834,-0.999834,-0.993686,-0.996742,-0.996984,-0.996996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN